In [1]:
import numpy as np
import pandas as pd # for loading data
import re # for removing special characters
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords # For removing stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nicktehrany/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nicktehrany/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
from util.helperfunctions import one_hot # Importing our Functions

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

## Creating a word list of all possible words from all Song Lyrics **TODO MAke this a function in util same for log regress**

In [6]:
corpus = []
lyrics_train = train_df['Lyrics']
sw = stopwords.words("english")
word_list = []
for i, value in train_df.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', str(lyrics_train[i])) # removes special characters
    text = text.lower() # lowercases everything
    text = text.split() # splits words
    text = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in text if not word in set(sw)]
    formatted_text = ""
    for word in text:
        if word not in word_list:
            word_list.append(word)
        formatted_text+=word+" "
    corpus.append(formatted_text)
    train_df['Lyrics'][i] = formatted_text
lyrics_train = corpus

#Cleaning variables to save some memory
del corpus, sw, text


## Creating one-hot vector for genres

In [7]:
from tensorflow.keras.utils import to_categorical
train_genres = []

for i, value in train_df.iterrows():
    genre = train_df['Genre'][i]
    index = 0
    if genre == '[\'Pop\']': index = 0
    elif genre == '[\'Rock\']': index = 1
    elif genre == '[\'Hip-Hop/Rap\']': index = 2
    elif genre == '[\'Country\']': index = 3
    elif genre == '[\'R&B/Soul\']': index = 4
    elif genre == '[\'Metal\']': index = 5
    elif genre == '[\'Alternative/Indie\']': index = 6
    elif genre == '[\'Folk\']': index = 7
    train_genres.append(index)

train_genres = to_categorical(train_genres, 8)

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [8]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list))
print(train_lyrics.shape)
lyrics_size = len(word_list)

#Cleaning variables to save some memory
# del train_df

(12240, 37818)


## One-Hot for test data **TODO More comments**

In [9]:
test_df = pd.read_csv("test.csv")
corpus = []
lyrics_test = test_df['Lyrics']
sw = stopwords.words("english")
for i, value in test_df.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', str(lyrics_test[i])) # removes special characters
    text = text.lower() # lowercases everything
    text = text.split() # splits words
    text = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in text if not word in set(sw)]
    formatted_text = ""
    for word in text:
        formatted_text+=word+" "
    corpus.append(formatted_text)
    value['Lyrics'] = formatted_text
lyrics_test = corpus

#Cleaning variables to save some memory
del corpus, sw, text

In [10]:
test_genres = []

for i, value in test_df.iterrows():
    genre = test_df['Genre'][i]
    index = 0
    if genre == '[\'Pop\']': index = 0
    elif genre == '[\'Rock\']': index = 1
    elif genre == '[\'Hip-Hop/Rap\']': index = 2
    elif genre == '[\'Country\']': index = 3
    elif genre == '[\'R&B/Soul\']': index = 4
    elif genre == '[\'Metal\']': index = 5
    elif genre == '[\'Alternative/Indie\']': index = 6
    elif genre == '[\'Folk\']': index = 7
    test_genres.append(index)

test_genres = to_categorical(test_genres, 8)

In [11]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size)
print(test_lyrics.shape)

#Cleaning variables to save some memory
# del test_df

(2160, 37818)


# Neural Network stuff 
## **Modify and run only the cells below to change the neural network**

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(units=128, input_shape=(lyrics_size,)))
model.add(Activation('relu'))            # activation layer
model.add(Dropout(0.8))
model.add(Dense(20))
model.add(Dense(8))
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               4840832   
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 168       
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
Tota

In [13]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.0001) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_lyrics, train_genres, epochs=5, validation_split=17/100, batch_size=16);

Train on 10159 samples, validate on 2081 samples
Epoch 1/5
10112/10159 [============================>.] - ETA: 0s - loss: 1.9395 - acc: 0.2527

## Testing the Neural Network's performace + Confusion Matrix

In [ ]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)

In [ ]:
def one_hot_reverse():
    genres = []
    genre_list = list(test_genres)

    for i in range(len(genre_list)):
        x = genre_list[i]
        for index in range(0, 8):
            if x[index] == 1.0:
                genres.append(index)

    return genres

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(test_lyrics)
cm=confusion_matrix(one_hot_reverse(),y_pred)
print(cm)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

## **Don't rerun the cell below, as this will result in adding dimesnions to the sets, which will not work! Can rerun all other cells except this one.**

In [ ]:
cnn_lyrics_train=lyrics_train[:, :, None]
cnn_lyrics_test=lyrics_test[:, :, None]

## Convolutional Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPool2D, Dropout, Flatten

model = Sequential()
model.add(Conv1D(128, kernel_size=(1), activation='relu', input_shape=(len(word_list),1)))
model.add(Conv1D(16, (1), activation='relu'))
model.add(Dropout(0.25)) # Dropout 25% of the nodes of the previous layer during training
model.add(Flatten())     # Flatten, and add a fully connected layer
model.add(Dense(32, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(cnn_lyrics_train, genre_train, epochs=5, batch_size=16, validation_split=1/10)

In [ ]:
results = model.evaluate(cnn_lyrics_test, genre_test, batch_size=16)